In [33]:
import yfinance as yf
import pandas as pd
import datetime as dt

# Define the parameters
ticker = 'SPY'  # Replace with the desired stock ticker
N = 120  # Number of months
x = 2  # Number of consecutive price rise or drop

# Fetch stock price data
end_date = dt.date.today()
start_date = end_date - pd.DateOffset(months=N)
data = yf.download(ticker, start=start_date, end=end_date)

# Define the function to identify x consecutive price rise or drop
def consecutive_rise_or_drop(prices, x):
    rise_count, drop_count = 0, 0

    for i in range(1, len(prices)):
        if prices[i] > prices[i - 1]:
            rise_count += 1
            drop_count = 0
        elif prices[i] < prices[i - 1]:
            drop_count += 1
            rise_count = 0
        else:
            rise_count, drop_count = 0, 0

        if rise_count == x:
            return 'rise'
        elif drop_count == x:
            return 'drop'

    return None

# Apply the trading strategy and calculate win rates
win_count_rise, win_count_drop = 0, 0
total_count_rise, total_count_drop = 0, 0

for i in range(x - 1, len(data) - 2):
    result = consecutive_rise_or_drop(data['Close'].iloc[i - x + 1 : i + 1].tolist(), x)

    if result == 'rise':
        total_count_drop += 1
        if data['Close'][i + 1] < data['Close'][i]:
            win_count_drop += 1
    elif result == 'drop':
        total_count_rise += 1
        if data['Close'][i + 1] > data['Close'][i]:
            win_count_rise += 1

win_rate_rise = win_count_rise / total_count_rise if total_count_rise > 0 else 0
win_rate_drop = win_count_drop / total_count_drop if total_count_drop > 0 else 0

print("Win rate betting on price rise after x consecutive drops: {:.2%}".format(win_rate_rise))
print("Win rate betting on price drop after x consecutive rises: {:.2%}".format(win_rate_drop))

[*********************100%***********************]  1 of 1 completed
Win rate betting on price rise after x consecutive drops: 0.00%
Win rate betting on price drop after x consecutive rises: 0.00%
